# MiniCPM-V 2.6

In [3]:
! python -m pip install --upgrade pip
! pip install -r MiniCPM_requirements.txt

  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 234.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 115.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 199.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 190.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 200.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 206.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 105.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 193.4 MB/s eta 0:00:00
Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.3.0
    Uninstalling triton-2.3.0:
      Successfully 

In [6]:
! pip install flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 327.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash_attn: filename=flash_attn-2.7.0.post2-cp310-cp310-linux_x86_64.whl size=182952710 sha256=3e3025f598ff80709ce2ee028707342731a29aa47da47a739ea3961105ef6157
  Stored in directory: /root/.cache/pip/wheels/bf/e3/ed/5e845387d52f2debd1bafb847bf3d774d3f0a3c8e31b1dc948
Successfully built flash_attn


In [8]:
import os
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, classification_report
from tqdm import tqdm
import json

# Open log file
log_path = './results/minicpm/'
os.makedirs(log_path, exist_ok=True)
log_file = os.path.join(log_path, 'prediction_log.txt')
log_file = open(log_file, 'w')

# Load data
df_table_prd = pd.read_csv('../LMM_sewerML/results/df_table_prd.csv')
df_table_dsc = pd.read_csv('../LMM_sewerML/results/df_table_dsc.csv')
image_dir = '../LMM_sewerML/images'

# Load MiniCPM model and tokenizer
model = AutoModel.from_pretrained('openbmb/MiniCPM-V-2_6', trust_remote_code=True, 
                                  attn_implementation='sdpa', torch_dtype=torch.bfloat16)
model = model.eval().cuda()
tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V-2_6', trust_remote_code=True)

# Define the prompt template
prompt_template = """
You are a virtual sewer technician with the capability to analyze images from CCTV cameras taken inside sewer pipes.
Your task is to examine each image and provide a concise, yet accurate, summary for retrieval. 
After summarizing, you must classify the image as defect types.
While providing the summary, remember the following guidelines:"
1) Provide a general overview of the image that you see, describing important elements such image clarity, lighting conditions, type of pipe (concrete, PVC, ...), presence of water.
2) Check for defects in the sewer pipes in the image.
3) Pipes in good condition usually show a smooth, unbroken surface, no visible signs of damage like cracks or collapses, and an absence of blockages such as roots.
4) On the other hand, you can have the following defects:
4a) Cracks, Breaks, and Collapses (RB): Identify visible cracks along the pipe, instances where the pipe has fractured or completely broken apart, and areas where the pipe has collapsed.
This includes longitudinal cracks, circumferential breaks, and complete structural failures that compromise the integrity of the sewer system.
4b) Surface Damage (OB): Detect areas of the pipe's interior that exhibit signs of wear, erosion, or damage on the surface.
This includes minor scratches, pitting, scaling, or any form of deterioration that affects the pipe's surface but does not necessarily penetrate deeply into the structure.
4c) Production Error (PF): Identify defects that originated during the pipe's manufacturing process, such as inconsistent pipe thickness, improper joint alignment, or material imperfections.
These are flaws that were introduced before installation and could potentially affect the pipe's performance or longevity.
4d) Deformations (DE): Recognize any alterations in the shape of the pipe, such as bending, sagging, or bulging, that indicate a deformation.
This includes both minor deformations that may affect flow efficiency and major deformations that threaten the pipe's structural integrity.
4e) Roots (RO): Detect the presence of roots infiltrating the sewer pipe, whether through joints, cracks, or other vulnerabilities.
This involves identifying both the initial stages of root intrusion and the more advanced stages where roots have significantly obstructed the pipe.
5) Additional considerations while analyzing the images: do not consider blurred text or user-defined circled areas in the images.
6) You will always try to describe the image that you see. Provide the output in JSON format as follows:

{ "DESCRIPTION": "<Description of the image that you see>", "CODE": "<Defect Code>"}

Note: The "CODE" can be selected from "RB", "OB", "PF", "DE", "RO". If no defect is detected, set "CODE" to "NoDefect".
"""

# Store predictions and actual labels
predictions = []
actual_labels = []
descriptions = []

# Add tqdm progress bar
for idx, row in tqdm(df_table_prd.iterrows(), total=len(df_table_prd), desc="Processing images"):
    img_id = row['img_id']
    ground_truth = row['defect_type']
    img_path = os.path.join(image_dir, img_id)
    
    # Check if image file exists
    if not os.path.exists(img_path):
        log_file.write(f"Image {img_id} not found. Skipping.\n")
        continue

    # Load image
    image = Image.open(img_path).convert('RGB')
    
    # Prepare the messages for MiniCPM
    question = prompt_template
    msgs = [{'role': 'user', 'content': [image, question]}]
    
    # Generate response
    try:
        res = model.chat(image=None, msgs=msgs, tokenizer=tokenizer)
        # Extract defect prediction and description from model output
        response_json = json.loads(res)
        description = response_json.get("DESCRIPTION", "No description provided")
        predicted_defect = response_json.get("CODE", "NoDefect")
    except json.JSONDecodeError:
        # Handle invalid JSON
        log_file.write(f"Invalid JSON response for Image ID {img_id}: {res}\n")
        description = "Error in JSON response"
        predicted_defect = "Error"

    # Append results to lists
    predictions.append(predicted_defect)
    actual_labels.append(ground_truth)
    descriptions.append(description)

    # Log intermediate outputs
    log_file.write(f"Image ID: {img_id} | Predicted: {predicted_defect} | Ground Truth: {ground_truth}\n")

# Calculate evaluation metrics
accuracy = accuracy_score(actual_labels, predictions)
f1 = f1_score(actual_labels, predictions, average='weighted')
report = classification_report(actual_labels, predictions)

log_file.write(f"\nAccuracy: {accuracy:.4f}\n")
log_file.write(f"F1 Score: {f1:.4f}\n")
log_file.write("\nClassification Report:\n")
log_file.write(report)

# Close log file
log_file.close()

# Save results to CSV files
df_table_prd['minicpm-v 2.6'] = predictions
df_table_prd.to_csv(log_path + 'df_table_prd.csv', index=False)

df_table_dsc['minicpm-v 2.6'] = descriptions
df_table_dsc.to_csv(log_path + 'df_table_dsc.csv', index=False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:653: UserWarning: Not enough free disk space to download the file. The expected file size is: 4932.75 MB. The target location /root/.cache/huggingface/hub/models--openbmb--MiniCPM-V-2_6/blobs only has 4.13 MB free disk space.
  warnings.warn(


model-00002-of-00004.safetensors:  96%|#########6| 4.75G/4.93G [00:00<?, ?B/s]

OSError: [Errno 28] No space left on device